In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess/dataset"
import os
import sys
file_path = "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess"
os.chdir(file_path)
__package__ == "trainer"

Mounted at /content/drive
__init__.py	  lm_dataset.py      __pycache__
lm_dataset.ipynb  pretrain_hq.jsonl  sft_mini_512.jsonl


False

In [ ]:
import argparse
import time
import math
import warnings
import torch
from torch import optim, nn
from contextlib import nullcontext
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
from dataset.lm_dataset import SFTDataset
from model.model_lora import load_lora, save_lora, apply_lora
warnings.filterwarnings('ignore')


# Logger function
def Logger(content):
    print(content)


def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model')
    sft_model_path = os.path.join(file_path, "./out/full_sft_512.pth")
    model = MiniMindForCausalLM(lm_config)
    state_dict = torch.load(sft_model_path, map_location = 'cuda')
    model.load_state_dict(state_dict)
    model.train()
    Logger(f'LLM可训练总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model.to('cuda')
    return model, tokenizer

In [ ]:
class Arguments:
  def __init__(self,
      out_dir = "./out",
      epochs = 10,
      batch_size = 32,
      learning_rate = 1e-4,
      device = "cuda" if torch.cuda.is_available() else "cpu",
      dtype = "bfloat16",
      accumulation_steps = 8,
      grad_clip = 1,
      warmup_iters = 0,
      log_interval = 100,
      save_interval = 100,
      local_rank = -1,
      hidden_size = 512,
      num_hidden_layers = 8,
      max_seq_len = 512,
      data_path = os.path.join(file_path, "./dataset/sft_mini_512.jsonl"),
      lora_name = "根据任务保存成lora_"
  ):
    self.out_dir = out_dir
    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.device = device
    self.dtype = dtype
    self.accumulation_steps = accumulation_steps
    self.grad_clip = grad_clip
    self.warmup_iters = warmup_iters
    self.log_interval = log_interval
    self.save_interval = save_interval
    self.local_rank = local_rank
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.max_seq_len = max_seq_len
    self.data_path = data_path
    self.save_dir = None
    self.tokens_per_iter = self.batch_size * self.max_seq_len
    self.lora_name = lora_name

In [ ]:
def train_epoch(epoch):
    loss_fct = nn.CrossEntropyLoss(reduction='none')
    start_time = time.time()
    for step, (X, Y, loss_mask) in enumerate(train_loader):
        X = X.to(args.device)
        Y = Y.to(args.device)
        loss_mask = loss_mask.to(args.device)
        lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch, args.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            res = model(X)
            loss = loss_fct(
                res.logits.view(-1, res.logits.size(-1)),
                Y.view(-1)
            ).view(Y.size())
            loss = (loss * loss_mask).sum() / loss_mask.sum()
            loss += res.aux_loss
            loss = loss / args.accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(lora_params, args.grad_clip)

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True)

        if step % args.log_interval == 0:
            spend_time = time.time() - start_time
            Logger(
                'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.12f} epoch_Time:{}min:'.format(
                    epoch + 1,
                    args.epochs,
                    step,
                    iter_per_epoch,
                    loss.item() * args.accumulation_steps,
                    optimizer.param_groups[-1]['lr'],
                    (spend_time)//60))

        if (step + 1) % args.save_interval == 0:
            model.eval()
            lora_save_path = f'./out/LoRA_weights_{lm_config.hidden_size}.pth'
            save_lora(model, lora_save_path)
            model.train()

In [ ]:
if __name__ == "__main__":
    args = Arguments()

    lm_config = MiniMindConfig(hidden_size=args.hidden_size, num_hidden_layers=args.num_hidden_layers)
    tokens_per_iter = args.batch_size * args.max_seq_len
    device_type = "cuda" if "cuda" in args.device else "cpu"

    ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast()
    base_seed = 1919810
    torch.manual_seed(base_seed)
    torch.cuda.manual_seed(base_seed)

    model, tokenizer = init_model(lm_config)
    apply_lora(model)

    total_params = sum(p.numel() for p in model.parameters())  # 总参数数量
    lora_params_count = sum(p.numel() for name, p in model.named_parameters() if 'lora' in name)  # LoRA 参数数量
    print(f"LLM 总参数量: {total_params}")
    print(f"LoRA 参数量: {lora_params_count}")
    print(f"LoRA 参数占比: {lora_params_count / total_params * 100:.2f}%")
    for name, param in model.named_parameters():
        if 'lora' not in name:
            param.requires_grad = False
    lora_params = []
    for name, param in model.named_parameters():
        if 'lora' in name:
            lora_params.append(param)

    # 只对 LoRA 参数进行优化
    optimizer = optim.AdamW(lora_params, lr=args.learning_rate)
    train_ds = SFTDataset(args.data_path, tokenizer, max_length=args.max_seq_len)
    train_sampler = DistributedSampler(train_ds) if ddp else None
    train_loader = DataLoader(
        train_ds,
        batch_size=args.batch_size,
        pin_memory=True,
        drop_last=False,
        shuffle=False
    )

    scaler = torch.cuda.amp.GradScaler(enabled=(args.dtype in ['float16', 'bfloat16']))
    iter_per_epoch = len(train_loader)

    for epoch in range(args.epochs):
        train_epoch(epoch)

LLM可训练总参数量：25.830 百万
LLM 总参数量: 25960960
LoRA 参数量: 131072
LoRA 参数占比: 0.50%


KeyboardInterrupt: 